## PPT八卦版-Jieba斷詞與Pandas資料整理

### 擷取資料

In [1]:
import time
import os
import requests
from bs4 import BeautifulSoup

resource_path = r'./res_gossiping'
if not os.path.exists(resource_path):
    os.mkdir(resource_path)

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
ss = requests.session()
ss.cookies["over18"] = "1"

url = "https://www.ptt.cc/bbs/Gossiping/index.html"

n = 10
for i in range(0,n):
    res = ss.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    article_title_html = soup.select('div[class="title"]')

    for each_article in article_title_html:
        try:
            each_article.a.text
            'https://www.ptt.cc' + each_article.a['href']
            article_url = 'https://www.ptt.cc' + each_article.a['href']
            article_text = each_article.a.text
            article_res = ss.get(article_url, headers=headers)
            article_soup = BeautifulSoup(article_res.text, 'html.parser')

            push_up = 0
            push_down = 0
            score = 0
            author = ''
            title = ''
            datetime = ''
            article_content = article_soup.select('div#main-content')[0].text.split('--')[0]
            push_info_list = article_soup.select('div[class="push"] span')
            for info in push_info_list:
                if '推' in info.text:
                    push_up += 1
                if '噓' in info.text:
                    push_down += 1
            article_info_list = article_soup.select('div[class="article-metaline"] span')
            for n, info in enumerate(article_info_list):
                if (n+1)%6 == 2:
                    author = info.text
                if (n+1)%6 == 4:
                    title = info.text
                if (n+1)%6 == 0:
                    datetime = info.text
            score = push_up - push_down
            article_content += '\n---split---\n'
            article_content += '推: %s\n'%(push_up)
            article_content += '噓: %s\n' % (push_down)
            article_content += '分數: %s\n'%(score)
            article_content += '作者: %s\n'%(author)
            article_content += '標題: %s\n'%(title)
            article_content += '時間: %s\n'%(datetime)
            try:
                new_article_text = article_text
                for iw in '[\/:*?"<>|]':
                    new_article_text = new_article_text.replace(iw, '_')
                with open(r'%s/%s.txt' % (resource_path, new_article_text), 'w', encoding='utf-8') as w:
                    w.write(article_content)
                #print()
            except FileNotFoundError as e:
                pass
            except OSError as e:
                pass

        except AttributeError as e:
            pass

    url = 'https://www.ptt.cc' + soup.select('div[class="btn-group btn-group-paging"]')[0].select('a')[1]['href']

### 開始斷詞

In [2]:
import jieba
import os

resource_path = r'./res_gossiping'
file_list = os.listdir(resource_path)
file_list[:5]

['Re_ _問卦_ 大家年終剩下多少？.txt',
 'Re_ _問卦_ 把小孩取名音同「賤民」「賤人」的父母.txt',
 'Re_ _問卦_ 整個過年都沒出門的+1.txt',
 'Re_ _問卦_ 早餐這樣590算盤嗎？.txt',
 'Re_ _問卦_ 最後生還者美劇的艾莉很不行.txt']

In [3]:
#將所有文章存成一個字串
all_article_str = ""
for article in file_list:
    article_path = resource_path + "/" + article
    with open(article_path, "r", encoding="utf-8") as f:
        temp_article = f.read().split("---split---")[0].split("\n")[1:]
    for article_line in temp_article:
        all_article_str += article_line + "\n"
print(all_article_str)

我阿肥啦

這魯肥略懂略懂

魯肥發給爸媽各6000

外公外婆各2000

除此之外

鞋子3700 罰單3000 刮刮樂4000

就沒了

歸零 嗚嗚嗚嗚嗚

大概是這樣

嘻嘻


※ 引述《elans73 (大俠毛一撮)》之銘言
: 各位大大們新年快樂
: 今天初四了
: 阿伯也應該出事了
: 想問問大家年終剩下多少呢？




華人自古有賤名好養活的傳統，認為名字難聽孩子比較不會夭折，所以才有什麼賤民、賤
人、食糞這些取名。給你參考一下。



不是在講樓主

但我是建議八卦版的某些人

覺得日本怎樣怎樣

台灣又很貴怎樣怎樣的

建議就別出門了

在家留著，錢收好，別被賺走了

外面訂房停車機票都很擁擠

希望這些人千萬不要出來跟我搶

感恩



愛吃的本地人路過

還是幫店家說一下話，那個價格是外送的價格（原文下面很多人已經將真的 menu 附在推文了），如果原本價格大家還是覺得貴那也沒辦法。

但重要的應該是讓大家認識到，想要外送，實際上的成本有多高，假設外送抽三成，店家不願意自己吸收，那價格就只能漲五成以上。

所以捨不得花就勤勞一點自己去買好嗎


※ 引述《rondox (一個人跳舞)》之銘言：
: 哈囉你各位最後生還者迷哥迷妹
: 昨晚看了最後生還者劇集
: 劇集的艾莉長得跟遊戲裏也差太多了吧
: http://i.imgur.com/zRNnPn1.jpg
: 可愛的艾莉的臉變成充滿邪氣
: 真的很不行啊？
: 大家覺得勒？

嘻嘻

本來就很難找到相似的長相 演技又好的

是說如果是金鋼狼的休傑克曼跟羅根裡面的小蘿莉
https://i.imgur.com/X0TxOQF.jpeg

大家可以嗎






※ 引述《stuchan (憨吉)》之銘言：
: 基於環保，所以在新北校園通app上面申請了電子繳費單
: 結果現在變成自己要定期上app確認有沒有繳費單
: 繳費完還要等至少兩週作業時間之後再去確認繳費狀態
: 確認完還要自己列印收據繳回學校給老師
: 這無紙化是不是根本廢到掉渣
: 不但沒有環保，根本在整家長不敢再生小孩
: 到底是無紙化還是無子化，有沒有八卦？


要列印收據是老師自己的問題吧
正常把收據的截圖用LINE傳給班上群組給老師看不就好了?
教育局規定的流程一定不會說還要列印收據給老師的


認真來說政府的無紙化已

In [4]:
#使用預設模式進行分詞
str_list = jieba.cut(all_article_str)
word_count = {}
for i in str_list:
    if i in word_count:
        word_count[i] += 1
    else:
        word_count[i] = 1
word_count

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\irene\AppData\Local\Temp\jieba.cache
Loading model cost 0.690 seconds.
Prefix dict has been built successfully.


{'我': 146,
 '阿肥': 12,
 '啦': 39,
 '\n': 4326,
 '這魯肥': 1,
 '略懂': 2,
 '魯肥': 1,
 '發給': 1,
 '爸媽': 2,
 '各': 5,
 '6000': 1,
 '外公': 1,
 '外婆': 1,
 '2000': 7,
 '除此之外': 1,
 '鞋子': 1,
 '3700': 1,
 ' ': 2027,
 '罰單': 1,
 '3000': 3,
 '刮': 21,
 '刮樂': 12,
 '4000': 1,
 '就': 114,
 '沒': 106,
 '了': 217,
 '歸零': 1,
 '嗚': 12,
 '大概': 5,
 '是': 224,
 '這樣': 30,
 '嘻嘻': 6,
 '※': 78,
 '引述': 12,
 '《': 22,
 'elans73': 1,
 '(': 70,
 '大俠': 1,
 '毛': 2,
 '一': 12,
 '撮': 1,
 ')': 68,
 '》': 22,
 '之銘言': 11,
 ':': 460,
 '各位': 25,
 '大大': 2,
 '們': 13,
 '新年': 32,
 '快樂': 12,
 '今天': 23,
 '初四': 14,
 '阿伯': 5,
 '也': 126,
 '應該': 21,
 '出事': 3,
 '想': 27,
 '問問': 4,
 '大家': 56,
 '年終': 3,
 '剩下': 7,
 '多少': 8,
 '呢': 20,
 '？': 182,
 '華人': 3,
 '自古': 2,
 '有': 281,
 '賤名': 1,
 '好': 65,
 '養活': 1,
 '的': 674,
 '傳統': 5,
 '，': 1142,
 '認為': 15,
 '名字': 10,
 '難': 6,
 '聽': 10,
 '孩子': 18,
 '比較': 9,
 '不會': 18,
 '夭折': 1,
 '所以': 34,
 '才': 24,
 '什麼': 36,
 '賤民': 2,
 '、': 349,
 '賤': 1,
 '人': 110,
 '食糞': 1,
 '這些': 12,
 '取名': 3,
 '。': 381,
 '給你': 2,
 '參考': 6,
 '一下': 

### 載入自定義詞典補足沒有被識別的詞

In [5]:
#測試自定義詞典
jieba.load_userdict("./mydict.txt")

#再使用預設模式進行分詞
str_list = jieba.cut(all_article_str)
word_count = {}
for i in str_list:
    if i in word_count:
        word_count[i] += 1
    else:
        word_count[i] = 1
word_count

{'我': 146,
 '阿肥': 12,
 '啦': 39,
 '\n': 4326,
 '這魯肥': 1,
 '略懂': 2,
 '魯肥': 1,
 '發給': 1,
 '爸媽': 2,
 '各': 5,
 '6000': 1,
 '外公': 1,
 '外婆': 1,
 '2000': 7,
 '除此之外': 1,
 '鞋子': 1,
 '3700': 1,
 ' ': 2027,
 '罰單': 1,
 '3000': 3,
 '刮刮樂': 12,
 '4000': 1,
 '就': 114,
 '沒': 106,
 '了': 217,
 '歸零': 1,
 '嗚': 12,
 '大概': 5,
 '是': 224,
 '這樣': 30,
 '嘻嘻': 6,
 '※': 78,
 '引述': 12,
 '《': 22,
 'elans73': 1,
 '(': 70,
 '大俠': 1,
 '毛': 2,
 '一': 12,
 '撮': 1,
 ')': 68,
 '》': 22,
 '之銘言': 11,
 ':': 460,
 '各位': 25,
 '大大': 2,
 '們': 13,
 '新年快樂': 7,
 '今天': 23,
 '初四': 14,
 '阿伯': 5,
 '也': 126,
 '應該': 21,
 '出事': 3,
 '想': 27,
 '問問': 4,
 '大家': 56,
 '年終': 3,
 '剩下': 7,
 '多少': 8,
 '呢': 20,
 '？': 182,
 '華人': 3,
 '自古': 2,
 '有': 281,
 '賤名': 1,
 '好': 65,
 '養活': 1,
 '的': 674,
 '傳統': 5,
 '，': 1142,
 '認為': 15,
 '名字': 10,
 '難聽': 1,
 '孩子': 18,
 '比較': 9,
 '不會': 18,
 '夭折': 1,
 '所以': 34,
 '才': 24,
 '什麼': 36,
 '賤民': 2,
 '、': 349,
 '賤': 1,
 '人': 110,
 '食糞': 1,
 '這些': 12,
 '取名': 3,
 '。': 381,
 '給你': 2,
 '參考': 6,
 '一下': 22,
 '不是': 26,
 '在講': 2,
 '樓

### 載入停用字字典移除無意義字詞

In [6]:
#停用字詞典(不需要用的詞)
stopword_path = "./stopword.txt"
stopword_list = []
with open(stopword_path, "r", encoding="utf-8") as f:
    for line in f.readlines():
        stopword_list.append(line.replace("\n",""))
#print(stopword_list)

#只留下長度>1且不在停用字詞典裡的詞
word_list = [(k, word_count[k]) for k in word_count if len(k)>1 and k not in stopword_list]
word_list

[('阿肥', 12),
 ('這魯肥', 1),
 ('略懂', 2),
 ('魯肥', 1),
 ('發給', 1),
 ('爸媽', 2),
 ('6000', 1),
 ('外公', 1),
 ('外婆', 1),
 ('2000', 7),
 ('除此之外', 1),
 ('鞋子', 1),
 ('3700', 1),
 ('罰單', 1),
 ('3000', 3),
 ('刮刮樂', 12),
 ('4000', 1),
 ('歸零', 1),
 ('大概', 5),
 ('這樣', 30),
 ('嘻嘻', 6),
 ('引述', 12),
 ('elans73', 1),
 ('大俠', 1),
 ('之銘言', 11),
 ('各位', 25),
 ('大大', 2),
 ('新年快樂', 7),
 ('今天', 23),
 ('初四', 14),
 ('阿伯', 5),
 ('應該', 21),
 ('出事', 3),
 ('問問', 4),
 ('大家', 56),
 ('年終', 3),
 ('剩下', 7),
 ('多少', 8),
 ('華人', 3),
 ('自古', 2),
 ('賤名', 1),
 ('養活', 1),
 ('傳統', 5),
 ('認為', 15),
 ('名字', 10),
 ('難聽', 1),
 ('孩子', 18),
 ('比較', 9),
 ('不會', 18),
 ('夭折', 1),
 ('所以', 34),
 ('什麼', 36),
 ('賤民', 2),
 ('食糞', 1),
 ('這些', 12),
 ('取名', 3),
 ('給你', 2),
 ('參考', 6),
 ('一下', 22),
 ('不是', 26),
 ('在講', 2),
 ('樓主', 1),
 ('建議', 4),
 ('八卦版', 5),
 ('某些', 2),
 ('覺得', 15),
 ('日本', 24),
 ('怎樣', 6),
 ('台灣', 62),
 ('出門', 1),
 ('在家', 4),
 ('留著', 1),
 ('收好', 1),
 ('外面', 2),
 ('訂房', 1),
 ('停車', 2),
 ('機票', 1),
 ('擁擠', 1),
 ('希望', 10),
 ('人千萬

In [7]:
#排序(哪些詞被談論較多次)
word_list.sort(key=lambda x:x[1], reverse=True)
word_list[:10]

[('新聞', 125),
 ('文章', 87),
 ('水桶', 74),
 ('八卦', 64),
 ('台灣', 62),
 ('媒體', 58),
 ('完整', 58),
 ('記者', 57),
 ('大家', 56),
 ('標題', 50)]

### 整理文章資訊

In [8]:
import pandas as pd

columns = ["推數", "噓數", "文章分數", "文章作者", "文章名稱", "發表時間"]

article_info = []
for article in file_list:
    article_path = resource_path + "/" + article
    with open(article_path, "r", encoding="utf-8") as f:
        temp_article = f.read().split("---split---")[1].split("\n")[1:-1]
    article_info.append(temp_article)
article_info[:3]

[['推: 4',
  '噓: 1',
  '分數: 3',
  '作者: mk203125 (戴奧辛)',
  '標題: Re: [問卦] 大家年終剩下多少？',
  '時間: Wed Jan 25 10:29:07 2023'],
 ['推: 2',
  '噓: 0',
  '分數: 2',
  '作者: remora (remora)',
  '標題: Re: [問卦] 把小孩取名音同「賤民」「賤人」的父母',
  '時間: Wed Jan 25 12:17:17 2023'],
 ['推: 0',
  '噓: 0',
  '分數: 0',
  '作者: elite9q ()',
  '標題: Re: [問卦] 整個過年都沒出門的+1',
  '時間: Wed Jan 25 10:26:29 2023']]

In [9]:
#整理成DataFrame
df = pd.DataFrame(data=article_info, columns=columns)
df.head()

,推數,噓數,文章分數,文章作者,文章名稱,發表時間
0,推: 4,噓: 1,分數: 3,作者: mk203125 (戴奧辛),標題: Re: [問卦] 大家年終剩下多少？,時間: Wed Jan 25 10:29:07 2023
1,推: 2,噓: 0,分數: 2,作者: remora (remora),標題: Re: [問卦] 把小孩取名音同「賤民」「賤人」的父母,時間: Wed Jan 25 12:17:17 2023
2,推: 0,噓: 0,分數: 0,作者: elite9q (),標題: Re: [問卦] 整個過年都沒出門的+1,時間: Wed Jan 25 10:26:29 2023
3,推: 4,噓: 0,分數: 4,作者: brucechen2 (雞頭),標題: Re: [問卦] 早餐這樣590算盤嗎？,時間: Wed Jan 25 11:28:47 2023
4,推: 5,噓: 0,分數: 5,作者: teeheehee (軟尾拋拋翔(′・ω・`)),標題: Re: [問卦] 最後生還者美劇的艾莉很不行,時間: Wed Jan 25 11:54:58 2023


In [10]:
#將:前面不需要的字詞移除
for c in df:
    df[c] = df[c].apply(lambda x : x.split(": ")[-1].strip())
df.head()

,推數,噓數,文章分數,文章作者,文章名稱,發表時間
0,4,1,3,mk203125 (戴奧辛),[問卦] 大家年終剩下多少？,Wed Jan 25 10:29:07 2023
1,2,0,2,remora (remora),[問卦] 把小孩取名音同「賤民」「賤人」的父母,Wed Jan 25 12:17:17 2023
2,0,0,0,elite9q (),[問卦] 整個過年都沒出門的+1,Wed Jan 25 10:26:29 2023
3,4,0,4,brucechen2 (雞頭),[問卦] 早餐這樣590算盤嗎？,Wed Jan 25 11:28:47 2023
4,5,0,5,teeheehee (軟尾拋拋翔(′・ω・`)),[問卦] 最後生還者美劇的艾莉很不行,Wed Jan 25 11:54:58 2023


In [11]:
#存成csv檔
df.to_csv("ptt_gossiping_article_info.csv", index=False, encoding="utf-8-sig")